<a href="https://colab.research.google.com/github/mdefilippi85/P-Project-Big-Data/blob/master/example%20geopandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
pip install simpledbf 

In [ ]:
from simpledbf import Dbf5

In [ ]:
dbf = Dbf5('/Budapest_553_PS_75_0.dbf')

In [ ]:
df = dbf.to_dataframe()

# New Section

In [ ]:
pip install geopandas

In [ ]:
import geopandas as gpd

In [ ]:
df_regioni = gpd.read_file(filename="/Budapest_553_PS_75_0.shp")

In [ ]:
df_regioni.tail()

In [ ]:
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

MessageError: ignored